In [1]:
import json
from web3 import Web3
from solc import compile_standard

In [3]:
compiled_sol = compile_standard({
    "language": "Solidity",
    "sources": {
        "Greeter.sol": {
            "content": '''
                pragma solidity ^0.8.0;

                contract Greeter {
                  string public greeting;

                  constructor() public {
                      greeting = 'Hello';
                  }

                  function setGreeting(string memory _greeting) public {
                      greeting = _greeting;
                  }

                  function greet() view public returns (string memory) {
                      return greeting;
                  }
                }
              '''
        }
    },
    "settings":
        {
            "outputSelection": {
                "*": {
                    "*": [
                        "metadata", "evm.bytecode"
                        , "evm.bytecode.sourceMap"
                    ]
                }
            }
        }
})

In [4]:
w3 = Web3(Web3.EthereumTesterProvider())

In [5]:
w3.eth.default_account = w3.eth.accounts[0]

In [6]:
bytecode = compiled_sol['contracts']['Greeter.sol']['Greeter']['evm']['bytecode']['object']

In [7]:
# get abi
abi = json.loads(compiled_sol['contracts']['Greeter.sol']['Greeter']['metadata'])['output']['abi']

In [8]:
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)

In [9]:
# Submit the transaction that deploys the contract
tx_hash = Greeter.constructor().transact()

In [11]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [12]:
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

In [13]:
greeter.functions.greet().call()

TransactionFailed: 